In [1]:
import pickle
import numpy
from sklearn.decomposition import NMF
import pandas as pd

In [2]:
import pandas as pd

In [3]:
d = pd.read_pickle("data/articles.pkl")

In [4]:
d

,document_type,web_url,lead_paragraph,abstract,snippet,news_desk,word_count,source,section_name,subsection_name,_id,pub_date,print_page,headline,content
0,article,http://www.nytimes.com/2013/10/03/sports/footb...,You would think that in a symmetric zero-sum s...,None,You would think that in a symmetric zero-sum s...,Sports,347,The New York Times,Sports,Pro Football,524d4e3a38f0d8198974001f,2013-10-03T00:00:00Z,None,Week 5 Probabilities: Why Offense Is More Impo...,the original goal building model football fore...
1,article,http://www.nytimes.com/2013/10/03/us/new-immig...,House Democrats on Wednesday unveiled an immig...,House Democrats unveil immigration bill that p...,House Democrats on Wednesday unveiled an immig...,National,83,The New York Times,U.S.,None,524cf71338f0d8198973ff7b,2013-10-03T00:00:00Z,21,New Immigration Bill Put Forward,house unveiled immigration bill provides path ...
2,article,http://www.nytimes.com/2013/10/03/us/arizona-j...,A federal judge on Wednesday ordered the appoi...,Federal Judge Murray Snow orders the appointme...,A federal judge on Wednesday ordered the appoi...,National,160,The New York Times,U.S.,None,524cf50e38f0d8198973ff79,2013-10-03T00:00:00Z,21,Arizona: Judge Orders Monitor to Oversee Maric...,federal judge wednesday ordered appointment in...
3,article,http://www.nytimes.com/2013/10/03/us/texas-sta...,Texas has turned to a compounding pharmacy to ...,"Documents show that Texas, nation's most activ...",Texas has turned to a compounding pharmacy to ...,National,112,The New York Times,U.S.,None,524cf39a38f0d8198973ff78,2013-10-03T00:00:00Z,21,Texas: State Bought Execution Drugs From a Com...,texas nation’s active death-penalty state turn...
4,article,http://www.nytimes.com/2013/10/03/sports/tenni...,"Rafael Nadal, aiming to end Novak Djokovic’s r...",None,"Rafael Nadal, aiming to end Novak Djokovic’s r...",Sports,49,The New York Times,Sports,Tennis,524cf28b38f0d8198973ff73,2013-10-03T00:00:00Z,14,Nadal on Track for No. 1 Spot,rafael nadal aiming end novak djokovic’s run 1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1400,article,http://www.nytimes.com/2013/10/01/us/politics/...,The federal government will shut down for the ...,"""Federal government will shut down for the fir...",The federal government will shut down for the ...,National,1227,The New York Times,U.S.,Politics,5249957e38f0d8198973f471,2013-10-01T00:00:00Z,1,Government Shuts Down in Budget Impasse,washington flurry last-minute move house senat...
1401,article,http://www.nytimes.com/2013/10/01/opinion/the-...,Al Shabab helps finance terrorist operations b...,None,Al Shabab helps finance terrorist operations b...,OpEd,781,International Herald Tribune,Opinion,None,524966da38f0d8198973f2ef,2013-10-01T00:00:00Z,,The White Gold of Jihad,how carnage upscale shopping mall nairobi link...
1402,article,http://www.nytimes.com/2013/10/01/world/middle...,After several explosions struck Iraq’s norther...,None,After several explosions struck Iraq’s norther...,Foreign,807,The New York Times,World,Middle East,524939c438f0d8198973f186,2013-10-01T00:00:00Z,,Once-Calm Area of Iraq Is Shaken by Bombings,baghdad several explosion suicide bomber struc...
1403,article,http://www.nytimes.com/2013/09/30/business/a-w...,Factories are finding that years of doing busi...,"""Manufacturers, attempting to meet rising dema...",Factories are finding that years of doing busi...,Business,2867,The New York Times,Business Day,None,5248d7ac38f0d8198973ee94,2013-09-30T00:00:00Z,1,A Wave of Sewing Jobs as Orders Pile Up at U.S...,minneapolis it past quitting time new textile ...


In [35]:
d.news_desk.unique()

array(['Sports', 'National', 'Business', 'Foreign', 'Letters',
       'Editorial', 'OpEd', 'Culture', 'Travel', 'Metro',
       'SundayBusiness', 'Arts&Leisure', 'Magazine', 'Weekend',
       'RealEstate', 'BookReview', 'NewsDesk', 'NODESK'], dtype=object)

In [36]:
len(d.news_desk.unique())

18

In [5]:
corpus = d['content']

In [6]:
corpus.head()

0    the original goal building model football fore...
1    house unveiled immigration bill provides path ...
2    federal judge wednesday ordered appointment in...
3    texas nation’s active death-penalty state turn...
4    rafael nadal aiming end novak djokovic’s run 1...
Name: content, dtype: object

## Clean up the texts
We'll use [`spacy`](https://spacy.io/usage/spacy-101) python package to preprocess the corpus.

In [7]:
!pip install spacy

In [8]:
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [9]:
import sys
import spacy
from string import punctuation
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

/home/libphy/anaconda3/envs/test/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


In [10]:
# # Load the spacy.en module if it hasn't been loaded already
# # When in ipython, execute the script using %run -i my_file.py to avoid
# # repeatedly loading in the english module

if not 'nlp' in locals():
    print("Loading English Module...")
    nlp = spacy.load("en_core_web_sm")

def lemmatize_string(doc, stop_words):
    # First remove punctuation form string
    # .translate works differently from 2 to 3 so check version number
    if sys.version_info.major == 3:
        PUNCT_DICT = {ord(punc): None for punc in punctuation}
        doc = doc.translate(PUNCT_DICT)
    else:
        # spaCy expects a unicode object
        doc = unicode(doc.translate(None, punctuation))

    # Run the doc through spaCy
    doc = nlp(doc)

    # Lemmatize and lower text
    tokens = [token.lemma_.lower() for token in doc]

    return ' '.join(w for w in tokens if w not in stop_words)

Loading English Module...


In [11]:
# Example of extending our STOPLIST
STOPLIST = set(list(ENGLISH_STOP_WORDS) + ["n't", "'s", "'m"])

processed = [lemmatize_string(doc, STOPLIST) for doc in corpus]

In [12]:
type(processed)

list

In [13]:
len(processed)

1405

In [14]:
type(processed[0])

str

In [15]:
processed[0]

'original goal building model football forecasting weigh importance facet game particular want know offense important defense defense really win championship " ’ important tricky question -pron- think symmetric zerosum sport like football offense defense equally important winning yard point gain offensive squad defensive squad surrender equal yard point ’ true game level aggregate squad performance team total number yard point gain surrender symmetric distribution offense wider word really good really bad offense average defense -pron- use advanced metric team performance like expect point add epa win probability add wpa measure spread performance standard deviation distribution tell u wide statistic distribute bell curve wide narrow 2000 season standard deviation epa 81 point team offense 65 point team defense wpa -pron- ’ 26 win team offense 20 win team defense measure 30 percent big offense defense lesson exercise team choose rank league ball average choose good offense -pron- expla

In [16]:
processed[1]

'house unveil immigration provide path citizenship 11 million immigrant live illegally tighten border security warn political fallout house republican fail act combine element senate pass june border security legislation win unanimous support homeland security committee house republican reject comprehensive approach question offer citizenship people break immigration law house unveil immigration provide path citizenship 11 million immigrant live illegally tighten border security warn political fallout house republican fail act combine element senate pass june border security legislation win unanimous support homeland security committee house republican reject comprehensive approach question offer citizenship people break immigration law'

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
c_train = ['Here is my corpus of text it says stuff and things','Here is some other document']
c_test = ['Yet another document','This time to test on']

tfidf = TfidfVectorizer()
tfidf.fit(c_train)

test_arr = tfidf.transform(c_test).todense()
print(tfidf.get_feature_names())

['and', 'corpus', 'document', 'here', 'is', 'it', 'my', 'of', 'other', 'says', 'some', 'stuff', 'text', 'things']


In [19]:
corpus = ['This is the first document.',
        'This document is the second document.',
        'And this is the third one.',
        'Is this the first document?']
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [24]:
type(X)

scipy.sparse.csr.csr_matrix

In [23]:
X.todense()

matrix([[0.        , 0.46979139, 0.58028582, 0.38408524, 0.        ,
         0.        , 0.38408524, 0.        , 0.38408524],
        [0.        , 0.6876236 , 0.        , 0.28108867, 0.        ,
         0.53864762, 0.28108867, 0.        , 0.28108867],
        [0.51184851, 0.        , 0.        , 0.26710379, 0.51184851,
         0.        , 0.26710379, 0.51184851, 0.26710379],
        [0.        , 0.46979139, 0.58028582, 0.38408524, 0.        ,
         0.        , 0.38408524, 0.        , 0.38408524]])

### Let's apply it to the corpus

In [27]:
tfidf = TfidfVectorizer()
tfidf.fit(processed)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [28]:
X = tfidf.fit_transform(processed)

In [29]:
X.shape

(1405, 34917)

In [34]:
X[:5,:10].todense() #mostly zero

matrix([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [45]:
tfidf.get_feature_names()[100:110]

['109000',
 '1093',
 '1095',
 '109955',
 '10day',
 '10foot',
 '10game',
 '10hole',
 '10inning',
 '10k']

In [48]:
tfidf.get_feature_names()[11200:11210]

['enter',
 'entered',
 'entering',
 'enterprise',
 'enterprising',
 'entertain',
 'entertainer',
 'entertaining',
 'entertainment',
 'enthral']

In [41]:
tfidf2 = TfidfVectorizer(ngram_range=(1,2)) #include bigram
X2 = tfidf2.fit_transform(processed)

In [42]:
X2.shape

(1405, 376259)

In [49]:
tfidf2.get_feature_names()[100:110]

['041',
 '041 percent',
 '045',
 '045 earn',
 '046',
 '046 correlation',
 '047',
 '047 percent',
 '05',
 '05 finish']

In [50]:
tfidf2.get_feature_names()[11200:11210]

['8000 yard',
 '80000',
 '80000 president',
 '80000 ticket',
 '800000',
 '800000 civilian',
 '800000 employee',
 '800000 federal',
 '800000 furlough',
 '800000 government']

In [51]:
from sklearn.decomposition import NMF

In [52]:
nmf = NMF(n_components=18, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(X2)

In [56]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [57]:
feat_names=tfidf2.get_feature_names()
no_top_words = 10
display_topics(nmf, feat_names, no_top_words)

pron say mr new year people state government work company
game season team pron player play league say yankee win
iran rouhani mr rouhani nuclear iranian mr netanyahu israel obama mr netanyahu
republican house senate shutdown government obama vote care health law
rivera yankee pettitte girardi jeter mariano pitch mariano rivera stadium yankee stadium
touchdown yard throw yard touchdown quarterback pass score interception giant game
percent bank rate market economy investor debt euro central bank bond
syria chemical weapon chemical weapon syrian resolution united united nation council security council
merkel ms merkel germany party german democrat europe ms social social democrat
oracle cup race america cup zealand new zealand team team new boat yacht
kenya mall shabab kenyan somalia nairobi attack militant somali official
hit wildcard inning run pitch inne homer game win ray
health insurance care exchange health care health insurance coverage plan law affordable
viewer million viewer r

In [58]:
d.news_desk.unique()

array(['Sports', 'National', 'Business', 'Foreign', 'Letters',
       'Editorial', 'OpEd', 'Culture', 'Travel', 'Metro',
       'SundayBusiness', 'Arts&Leisure', 'Magazine', 'Weekend',
       'RealEstate', 'BookReview', 'NewsDesk', 'NODESK'], dtype=object)